In [2]:
!pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 50.1 MB/s eta 0:00:00


In [4]:
import requests
from bs4 import BeautifulSoup
import mysql.connector

def get_website_title(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        title = soup.title.string if soup.title else "No title found"
        return title
    except Exception as e:
        print(f"Error fetching title for {url}: {e}")
        return None

def save_to_mysql(site_name, title):
    conn = None  # Initialize conn to None
    try:
        # Connect to MySQL
        conn = mysql.connector.connect(
        host='127.0.0.1',  # Use 127.0.0.1 instead of localhost if needed
        port=3306,         # Replace with your MySQL server's port
        user='root',
        password='password',
        database='websites'
)
        cursor = conn.cursor()
        # Create table if it doesn't exist
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS website_titles (
                id INT AUTO_INCREMENT PRIMARY KEY,
                site_name VARCHAR(255) NOT NULL,
                title TEXT NOT NULL
            )
        ''')

        # Insert site name and title into the table
        cursor.execute('''
            INSERT INTO website_titles (site_name, title)
            VALUES (%s, %s)
        ''', (site_name, title))

        conn.commit()
        print("Data saved successfully!")
    except mysql.connector.Error as err:
        print(f"MySQL Error: {err}")
    finally:
        if conn and conn.is_connected(): # Check if conn is not None and is connected
            cursor.close()
            conn.close()
def main():
    site_name = input("Enter the website URL (e.g., https://example.com): ")
    title = get_website_title(site_name)
    if title:
        save_to_mysql(site_name,title)